# Decentralized Autonomous Organizations (DAOs) -- Data gathering

**[Johnnatan Messias](https://johnnatan-messias.github.io/), August 2024**

This code gathers voting history of decentralized governance protocols.


In [1]:
import os
from web3 import Web3
import requests as re
import gzip
import pickle

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
from ethereum import to_checksum_address, get_contract, get_all_events_from_contract, get_blocks, get_batch_intervals

In [4]:
# Set directory paths
data_dir = os.path.abspath(
    os.path.join(os.getcwd(), "..", "data")) + os.sep
plots_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "plots")) + os.sep

blocks_dir = os.path.abspath(os.path.join(data_dir, "blocks")) + os.sep
# Create directories if they don't exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(blocks_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [5]:
block_max_ethereum = 20_563_000+1
contract_settings = [
    # Compound contracts
    {
        'contract_name': 'compound-governor-alpha',
        'contract_address': to_checksum_address('0xc0da01a04c3f3e0be433606045bb7017a7323e38'),
        'abi_address': None,
        'start': 9_601_459-1,
        'end': block_max_ethereum,
        'events': None},
    {
        'contract_name': 'compound-governor-bravo',
        'contract_address': to_checksum_address('0xc0da02939e1441f497fd74f78ce7decb17b66529'),
        'abi_address': None,
        'start': 12_006_099-1,
        'end': block_max_ethereum,
        'events': None},
    {
        'contract_name': 'compound-token',
        'contract_address': to_checksum_address('0xc00e94cb662c3520282e6f5717214004a7f26888'),
        'abi_address': None,
        'start': 9_601_359-1,
        'end': block_max_ethereum,
        'events': None},

    # Uniswap contracts
    {
        'contract_name': 'uniswap-governor-alpha-1',
        'contract_address': to_checksum_address('0xc4e172459f1e7939d522503b81afaac1014ce6f6'),
        'abi_address': None,
        'start': 12_543_659-1,
        'end': 13_059_342,
        'events': None},
    {
        'contract_name': 'uniswap-governor-alpha-2',
        'contract_address': to_checksum_address('0xc4e172459f1e7939d522503b81afaac1014ce6f6'),
        'abi_address': None,
        'start': 13_059_344-1,
        'end': block_max_ethereum,
        'events': None},
    {
        'contract_name': 'uniswap-governor-bravo',
        'contract_address': to_checksum_address('0x408ed6354d4973f66138c91495f2f2fcbd8724c3'),
        'abi_address': None,
        'start': 13_059_157-1,
        'end': block_max_ethereum,
        'events': None},
    {
        'contract_name': 'uniswap-token',
        'contract_address': to_checksum_address('0x1f9840a85d5af5bf1d1762f925bdaddc4201f984'),
        'abi_address': None,
        'start': 10_861_674-1,
        'end': block_max_ethereum,
        'events': None},
]

In [6]:
# This code connects to Paradigm Reth archive node (see https://x.com/gakonst/status/1702389827390546071)
eth_node = 'http://69.67.151.138:8545'

adapter = re.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=20)
session = re.Session()
session.mount('http://', adapter)
session.mount('https://', adapter)

w3 = Web3(Web3.HTTPProvider(eth_node, session=session,
          request_kwargs={'timeout': 60}))

print("Is connected to Ethereum node: ", w3.is_connected())
print("The most recent block is: ", w3.eth.block_number)

Is connected to Ethereum node:  True
The most recent block is:  20634589


In [7]:
contracts = {}
for contract_setting in contract_settings:
    contracts[contract_setting['contract_name']] = get_contract(
        w3,
        contract_setting['contract_address'],
        abi_contract_address=contract_setting['abi_address'], is_zksync=False)

In [8]:
for contract in contracts:
    print(contract)
    for event in contracts[contract].events:
        print('\t', event.event_name)

compound-governor-alpha
	 ProposalCanceled
	 ProposalCreated
	 ProposalExecuted
	 ProposalQueued
	 VoteCast
compound-governor-bravo
	 NewAdmin
	 NewImplementation
	 NewPendingAdmin
	 ProposalCanceled
	 ProposalCreated
	 ProposalExecuted
	 ProposalQueued
	 ProposalThresholdSet
	 VoteCast
	 VotingDelaySet
	 VotingPeriodSet
compound-token
	 Approval
	 DelegateChanged
	 DelegateVotesChanged
	 Transfer
uniswap-governor-alpha-1
	 ProposalCanceled
	 ProposalCreated
	 ProposalExecuted
	 ProposalQueued
	 VoteCast
uniswap-governor-alpha-2
	 ProposalCanceled
	 ProposalCreated
	 ProposalExecuted
	 ProposalQueued
	 VoteCast
uniswap-governor-bravo
	 NewAdmin
	 NewImplementation
	 NewPendingAdmin
	 ProposalCanceled
	 ProposalCreated
	 ProposalExecuted
	 ProposalQueued
	 ProposalThresholdSet
	 VoteCast
	 VotingDelaySet
	 VotingPeriodSet
uniswap-token
	 Approval
	 DelegateChanged
	 DelegateVotesChanged
	 MinterChanged
	 Transfer


## Gathering Events


In [ ]:
# Gather all contract events from contract_settings
for contract_setting in contract_settings:
    print('====== ', contract_setting['contract_name'], ' ======')
    batch_size = 2500 if not 'token' in contract_setting['contract_name'] else 50
    events = get_all_events_from_contract(contracts[contract_setting['contract_name']],
                                          start_block=contract_setting['start'],
                                          end_block=contract_setting['end'],
                                          batch_size=batch_size,
                                          max_workers=10,
                                          events=contract_setting['events'])
    with gzip.open(data_dir + 'events_' + contract_setting['contract_name'] + '.pkl.gz', 'wb') as f:
        pickle.dump(events, f)

## Gathering Transactions Related to VoteCast Events Trigerred by Compound and Uniswap


In [9]:
from ethereum import get_transactions

In [10]:
event_of_interest = ['VoteCast']
files_of_interest = ['compound-governor-alpha', 'compound-governor-bravo',
                     'uniswap-governor-alpha-1', 'uniswap-governor-bravo']
transactions_hashes = []
for file in files_of_interest:
    file_dir = os.path.join(data_dir, 'events_' + file + '.pkl.gz')
    with gzip.open(file_dir, 'rb') as f:
        events = pickle.load(f)
        transactions_hashes += [event['transactionHash'].hex()
                                for event in events['VoteCast']]
transactions_hashes = list(set(transactions_hashes))
print("There are", len(transactions_hashes), "transactions of interest")

There are 65902 transactions of interest


In [11]:
txs_data = get_transactions(
    w3=w3, txs_hashes=transactions_hashes, max_workers=20)

Gathering transactions...:   0%|          | 0/65902 [00:00<?, ?it/s]

In [12]:
file_dir = os.path.join(data_dir, 'transactions_of_votes.pkl.gz')
with gzip.open(file_dir, 'wb') as f:
    pickle.dump(txs_data, f)